## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [6]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBSSB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1387
+------------------+-----------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+--------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|          coeff_2|           coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|             res_4|            res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-----------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+--------------------+-----------+-----------+------+--------------------+------+
|-353.3940981424289|581.9572112045811| 213.2993765921791|192.05799222883283|

In [7]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [51]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='elevation, year, coeff_1,coeff_2,coeff_3,coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT elevation, year, coeff_1,coeff_2,coeff_3,coeff_4 FROM weather


elevation    year     coeff_1     coeff_2     coeff_3     coeff_4
0      805.0  1915.0 -353.394098  581.957211  213.299377  192.057992
1      805.0  1941.0  545.233221    3.992893  -78.235134 -168.573810
2      805.0  1942.0  692.195831   -6.845876  -57.277743 -270.096543
3      805.0  1943.0  728.558917   86.969530  124.238777   70.966886
4      805.0  1944.0  602.472357 -118.310534  -60.759922 -143.267354

In [54]:
year_station_table=pdf.pivot(index='year', columns='elevation', values='coeff_4')
year_station_table.head(10)

elevation  426.7      427.6      432.5   434.3   455.7   464.8   470.9   \
year                                                                      
1901.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1902.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1903.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1904.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1905.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1906.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1907.0        NaN  52.815216        NaN     NaN     NaN     NaN     NaN   
1908.0        NaN        NaN        NaN     NaN     NaN     NaN     NaN   
1909.0        NaN        NaN -49.445640     NaN     NaN     NaN     NaN   
1910.0        NaN        NaN  24.177011     NaN     NaN     NaN     NaN   

elevation  573.0   603.8       634.0    ...        1151.2      1271.0  1374.6  \
year                                    ...                                     
1901.0        NaN     NaN    7.084459   ...           NaN         NaN     NaN   
1902.0        NaN     NaN  -69.921032   ...           NaN         NaN     NaN   
1903.0        NaN     NaN    9.924186   ...           NaN         NaN     NaN   
1904.0        NaN     NaN  136.297444   ...    118.539156  241.443063     NaN   
1905.0        NaN     NaN  -10.912893   ...           NaN  149.985202     NaN   
1906.0        NaN     NaN -118.935994   ...   -149.121385   -9.958312     NaN   
1907.0        NaN     NaN   70.662197   ...     30.103660         NaN     NaN   
1908.0        NaN     NaN  -74.236882   ...    -45.922052         NaN     NaN   
1909.0        NaN     NaN         NaN   ...    179.671420         NaN     NaN   
1910.0        NaN     NaN         NaN   ...           NaN         NaN     NaN   

elevation  1400.6      1403.0      1478.3      1556.0  1950.7  2073.9  2425.0  
year                                                                           
1901.0        NaN  143.359967  239.894171         NaN     NaN     NaN     NaN  
1902.0        NaN  -13.416239   81.946578         NaN     NaN     NaN     NaN  
1903.0        NaN  178.429166  -52.998345         NaN     NaN     NaN     NaN  
1904.0        NaN   15.653673   77.408057         NaN     NaN     NaN     NaN  
1905.0        NaN -125.709465         NaN         NaN     NaN     NaN     NaN  
1906.0        NaN -123.867532         NaN         NaN     NaN     NaN     NaN  
1907.0        NaN         NaN         NaN         NaN     NaN     NaN     NaN  
1908.0        NaN         NaN         NaN         NaN     NaN     NaN     NaN  
1909.0        NaN         NaN         NaN         NaN     NaN     NaN     NaN  
1910.0        NaN         NaN         NaN  195.694921     NaN     NaN     NaN  

[10 rows x 49 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [53]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))
def MS(Mat):
    return np.nanmean(Mat**2)
def frac(M1,M2):
    N1=np.sqrt(np.nanmean(M1**2))
    N2=np.sqrt(np.nanmean(M2**2))
    return 100.0*(N2-N1)/N2
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

MS
print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station),"fraction explained", frac(tbl_minus_station,year_station_table)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year),"fraction explained", frac(tbl_minus_year,year_station_table)
print ""
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_elevation=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_elevation = year_station_table-mean_by_elevation

MS
print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_elevation),"fraction explained", frac(tbl_minus_station,year_station_table)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year),"fraction explained", frac(tbl_minus_year,year_station_table)
print ""
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_elevation)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total MS                   =  635957.627506
MS removing mean-by-station=  269090.34058 fraction explained 34.9518478438
MS removing mean-by-year   =  588020.266344 fraction explained 3.84274501268

total RMS                   =  797.469515095
RMS removing mean-by-station=  518.739183579
RMS removing mean-by-year   =  766.824795076
total MS                   =  635957.627506
MS removing mean-by-station=  269090.34058 fraction explained 34.9518478438
MS removing mean-by-year   =  588020.266344 fraction explained 3.84274501268

total RMS                   =  797.469515095
RMS removing mean-by-station=  518.739183579
RMS removing mean-by-year   =  766.824795076


In [55]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 194.088514151
0 after removing mean by year    = 148.865026623
0 after removing mean by stations= 128.886661456
1 after removing mean by year    = 127.796184348
1 after removing mean by stations= 127.635508465
2 after removing mean by year    = 127.595312705
2 after removing mean by stations= 127.583046267
3 after removing mean by year    = 127.578962854
3 after removing mean by stations= 127.577539524
4 after removing mean by year    = 127.577030203
4 after removing mean by stations= 127.576845094
